# RESOURCE PAPER FOLLOW
http://www.fao.org/docrep/X0490E/x0490e06.htm#TopOfPage

 ## UTILITIES
methods and functions

In [16]:
def celcius_to_kelvin(celcius):
    return celcius + 273.16

def kelvin_to_celcius(kelvin):
    return kelvin - 273.16   

def MJ_per_m2_day_to_mm_per_day(MJ):
    return MJ * 0.408

def mm_per_day_to_MJ_per_m2_day(mm):
    return mm / 0.408

 # STEP # 1
 ## VARIABLES AND CONSTANTS

In [17]:
# aerodynamic_resistance
measurements_height = 4

crop_height_ref = 0.12 # [m]
wind_speed_measurement = 1 # MEASUREMENT [m / s]

wind_speed_measurement_height = measurements_height # [m] 
air_rh_measurement_height = measurements_height # [m]
air_temp_measurement_height = measurements_height # [m]
KARMAN_CONSTANT = 0.41 # [None]

# surface_resistance
STOMATAL_RESISTANCE = 100 # [s / m] plant surface openness to disipate water vapor - the lost of water irrigated throught the plant's veins

# average_atmospheric_pressure
elevation_above_sea_level = 1800 # [m] MEASUREMENT 

# psychrometric_constant
VAPORIZATION_LATENT_HEAT = 2.45 # [MJ / kg] # This is the latent heat for an air temperature of about 20°C.
atmospheric_pressure = 1 # [kPa]
SPECIFIC_AIR_HEAT_AT_CONSTANT_PRESSURE = 1.013e-3 # [MJ / kg * Celcius]
RATIO_MOLECULAR_WEIGHT_WATER_VAPOUR_AND_DRY_AIR = 0.622 # [None] is a ratio - no units

# air_temp_mean
air_temp_max = 25 # [°C] -> converted to kelvin
air_temp_min = 18 # [°C] -> converted to kelvin

# actual_vapour_pressure
air_rhumidity_max = 82 # [%]
air_rhumidity_min = 54 # [%]
# actual_vapour_pressure_rh
from statistics import mean 
air_rhumidity_mean = mean([air_rhumidity_max, air_rhumidity_min])

# STEP 3 - RADIATION
# extraterrestrial_radiation (Ra)
from datetime import datetime
from math import radians
# day_of_the_year = 154 # [day] between 1 - 365
day_of_the_year = int(datetime(2017,5,15).strftime('%j')) # [day] between 1 - 365
latitude = radians(-22.54) # [°] coordinates, location global position , neede in radians
SOLAR_CONSTANT = 0.082 # [MJ / pow(m, 2) * min]

# global_radiation
measured_daylight_hour_duration = 7.1 # amounth of daylight hours in a day.
REGRESSION_CONSTANT = 0.25 # only when there is no solar radiation data, calculated like this - expressing the fraction of extraterrestrial radiation reaching the earth on overcast days (n or measured_daylight_length = 0)
FRACTION_EXTRATERRESTRIAL_RADIATION = 0.50 # fraction of extraterrestrial radiation reaching the earth on clear days (n = N) or (measured_daylight_length = max_daylight_length)

# net solar o shortwave radiation
albedo = 0.23 # reflection factor - photons refelcted back due to a reflective surface like snow, mirror, water etc.

# longwave radiation
STEFAN_BOLTZMANN_CONSTANT = 4.903e-9 # [4.903e-9 MJ / pow(K, 4) * pow(m, 2) * day]

# soil_flux
air_temp_previous_month = 14.1 # Celcius
air_temp_current_month = 16.1 # Celcius
air_temp_next_month = 18.8 # Celcius

 # STEP # 2
 
 ## WEATHER ANALYSIS
 ---

* **ALTITUDE** abovve sea level
* **LATITUDE & LONGITUDE** location
* **TEMPERATURE** average, daily min and max temperature using mean air temperature instead of maximum and minimum air temperatures yields a lower saturation vapour pressure es, and hence a lower vapour pressure difference (es - ea), and a lower reference evapotranspiration estimate.
* **HUMIDITYY** with psychomretric or dewpoint temp or min and max relative humidity
* **WIND SPEED** at 2 meter above soil, if at different height a modification most be calculated

In [18]:
# ATMOSPHERIC PRESSURE [kPa]
    # z = elevation above sea level [m] meters
average_atmospheric_pressure = 101.3 * pow(((293 - 0.0065 * elevation_above_sea_level) / 293), 5.26) # EQUATION 7
average_atmospheric_pressure

81.75579640764421

In [19]:
# PSYCHROMETRICS CONSTANT [kPa / °C]
    # γ = psychrometric constant [kPa / °C]
     # λ = latent heat of vaporization, 2.45 [MJ / kg]
     # Cp = specific heat at constant pressure, 1.013 10-3 [MJ / kg * °C]
     # ε = ratio molecular weight of water vapour/dry air = 0.622
     # average_atmospheric_pressure = calculated befiore [kPa]
psychrometric_constant = (SPECIFIC_AIR_HEAT_AT_CONSTANT_PRESSURE * average_atmospheric_pressure) / (RATIO_MOLECULAR_WEIGHT_WATER_VAPOUR_AND_DRY_AIR * VAPORIZATION_LATENT_HEAT)  # EQUATION 8
# psychrometric_constant_two = 0.665e-3 * average_atmospheric_pressure # EQUATION 8 (the same as previous but accuracy is lost with generalization of decimals)
psychrometric_constant # [kPa / °C]

0.054346493707555336

In [20]:
# AIR TEMPERATURE [°C]
    # mean temperature is the average of the max and min temnp throghout a 24h period - when calculating it might be a list mof average max and min then to take the mean since is a range of 10 days or longer
	# temperature in KELVIN
air_temp_mean = mean([air_temp_max, air_temp_min]) # EQUATION 9
air_temp_mean_in_kelvin = celcius_to_kelvin(air_temp_mean)
air_temp_mean_in_kelvin
air_temp_mean

21.5

In [21]:
# MEAN SATURATION VAPOUR PRESSURE (es) [kPa]
	# As saturation vapour pressure is related to air temperature, it can be calculated from the air temperature. The relationship is expressed by
	# eo(T) saturation vapour pressure at the air temperature T [kPa]
	# air_temp = air temperature [°C]
	# exp[..] 2.7183 (base of natural logarithm) raised to the power [..].
# //TODO check if eo formula is the right way to calculate this formula
from math import expm1, exp
eo_temp_max = 0.6108 * exp((17.27 * air_temp_max) / (air_temp_max + 237.3)) # EQUATION 11 # function based on temperature
eo_temp_min = 0.6108 * exp((17.27 * air_temp_min) / (air_temp_min + 237.3)) # EQUATION 11 # function based on temperature
# eo_temp_mean = 0.6108 * exp((17.27 * air_temp_mean_in_celcius) / (air_temp_mean_in_celcius + 237.3)) # EQUATION 11 # function based on temperature
saturation_vapour_pressure = (eo_temp_max + eo_temp_min) / 2 # EQUATION 12
# saturation_vapour_pressure_temp_mean = (eo_temp_mean + eo_temp_mean) / 2 # EQUATION 12
saturation_vapour_pressure

2.6158834600836665

In [22]:
# SLOPE OF SATURATION VAPOUR PRESSURE CURVE (∆) [None]
    # For the calculation of evapotranspiration, the slope of the relationship between saturation vapour pressure and
    # temperature, ∆, is required. The slope of the curve (Figure 11) at a given temperature is given by.
    # ∆ = slope of saturation vapour pressure curve at air temperature T [kPa / °C],
    # air_temp = air temperature [°C],
    # exp[..] 2.7183 (base of natural logarithm) raised to the power [..].
    # Values of slope ∆ for different air temperatures are given in Annex 2 (Table 2.4). In the FAO Penman-Monteith
    # equation, where ∆ occurs in the numerator and denominator, the slope of the vapour pressure curve is
    # calculated using mean air temperature.
saturation_vapour_pressure_slope = (4098 * 0.6108 * exp((17.27 * air_temp_mean) / (air_temp_mean + 237.3))) / (pow((air_temp_mean + 237.3), 2)) # EQUATION 13
saturation_vapour_pressure_slope

0.15690345906391895

In [23]:
# this calculation has many errors, but to measure with more reliability, the dewpoint temperature is needed or the dry bulb temp andd wet bulb temp thta is measuremen with another sensor.
# ACTUAL VAPOUR PRESSURE (ea) DERIVED FROM RELATIVE HUMIDITY DATA [kPa]
    # ea = actual vapour pressure [kPa]
    # eo_T_min = saturation vapour pressure at daily minimum temperature [kPa]
    # eo_T_max = saturation vapour pressure at daily maximum temperature [kPa]
    # RH_max = maximum relative humidity [%]
    # RH_min = minimum relative humidity [%]

# --for RH_max and RH_min
actual_vapour_pressure_rh = (eo_temp_min * (air_rhumidity_max / 100) + eo_temp_max * (air_rhumidity_min / 100)) / 2 # EQUATION 17
actual_vapour_pressure_rh

1.7015355568176478

In [24]:
# --for RH_mean -- In the absence of RH max and RH min , another equation can be used to estimate e a
    # RH_mean = the mean relative humidity, defined as the average between RH max and RH min.

actual_vapour_pressure_rh_mean = (air_rhumidity_mean / 100) * ((eo_temp_max + eo_temp_min) / 2) # EQUATION 19
actual_vapour_pressure_rh_mean

1.7788007528568932

In [25]:
# VAPOUR PRESSURE DEFICIT (es - ea) [kPa]
    # The vapour pressure deficit is the difference between the saturation (es) and actual vapour pressure (ea) for a
    # given time period. For time periods such as a week, ten days or a month e s is computed from Equation 12 using
    # the T max and T min averaged over the time period and similarly the e a is computed with one of the equations 4 to
    # 19, using average measurements over the period.
vapour_pressure_deficit = saturation_vapour_pressure - actual_vapour_pressure_rh
vapour_pressure_deficit

0.9143479032660187

# STEP # 3
 
## SUN ANALYSIS
---

* **EXTRATERRESTRIAL RADIATION**  radiation that reacvh atmosphere [MJ / m2 * day]
* _daily or shorter periods measurements_

In [26]:
# EXTRATERRESTRIAL RADIATION (Ra) [MJ / pow(m, 2) * day]
from math import pi, sin, cos, tan, acos
    # day_of_the_year = is the number of the day in the year between 1 (1 January) and 365 or 366 (31 December)
inverse_rdistance_earth_sun = 1 + 0.033 * cos(((2 * pi) / 365) * day_of_the_year) # EQUATION 23
solar_declination = 0.409 * sin(((2 * pi) / 365) * day_of_the_year - 1.39) # EQUATION 24
sunset_hour_angle = acos(-tan(latitude) * tan(solar_declination)) # EQUATION 25
    # Ra = extraterrestrial radiation [MJ / m * day]
    # dr = inverse relative distance Earth-Sun (Equation 23)
    # φ = latitude [rad] (Equation 22)
    # Gsc = solar constant = 0.0820 MJ m min
    # ωs = sunset hour angle (Equation 25 or 26) [rad]
    # δ = solar decimation (Equation 24) [rad]
extraterrestrial_radiation = ((24 * 60) / pi) * SOLAR_CONSTANT * inverse_rdistance_earth_sun * (sunset_hour_angle * sin(latitude) * sin(solar_declination) + cos(latitude) * cos(solar_declination) * sin(sunset_hour_angle)) # EUQATION 21
extraterrestrial_radiation

25.29268412905211

In [27]:
# the water lost in mm / day (water depths units) due to extraterrestrial radiation. that is if only the radiation that reach the atmosphere would hit the ground on earth
extraterrestrial_radiation_mm = MJ_per_m2_day_to_mm_per_day(extraterrestrial_radiation)
extraterrestrial_radiation_mm

10.31941512465326

In [28]:
# EXTRATERRESTRIAL RADIATION FOR HOURLY OR SHORTER PERIODS (Ra)
    # t = standard clock time at the midpoint of the period [hour]. For example for a period between 14.00 and 15.00 hours, t = 14.5,
    # Lz = longitude of the centre of the local time zone [degrees west of Greenwich]. For example, Lz = 75, 90, 105 and 120° for the Eastern, Central, Rocky Mountain and Pacific time zones (United States) and Lz = 0° for Greenwich, 330° for Cairo (Egypt), and 255° for Bangkok (Thailand),
    # Lm = longitude of the measurement site [degrees west of Greenwich],
    # Sc = seasonal correction for solar time [hour].
    # Of course, ω < -ω s or ω > ω s from Equation 31 indicates that the sun is below the horizon so that, by definition, Ra is zero.
ω = (pi / 12) * ((t + 0.066667 * (Lz - Lm) + Sc) - 12) # EQUATION 31
    # day_of_the_year = is the number of the day in the year between 1 (1 January) and 365 or 366 (31 December)
b = (2 * pi * (day_of_the_year - 81)) / 364 # EQUATION 33
Sc = 0.1645 * sin(2b) - 0.1255 * cos(b) - 0.025 * sin(b) # EQUATION 32
    # ω = solar time angle at midpoint of hourly or shorter period [rad],
    # t1 = length of the calculation period [hour]: i.e., 1 for hourly period or 0.5 for a 30-minute period.
ωs1 = ω - ((pi * t1) / 24) # EQUATION 29
ωs2 = ω + ((pi * t1) / 24) # EQUATION 30
    # Ra = extraterrestrial radiation in the hour (or shorter) period [MJ / pow(m, 2) * hour]
    # Gsc = solar constant = 0.0820 MJ / pow(m, 2) * min
    # dr = inverse relative distance Earth-Sun (Equation 23)
    # δ = solar declination [rad] (Equation 24)
    # φ = latitude [rad] (Equation 22)
    # ωs1 = solar time angle at beginning of period [rad] (Equation 29)
    # ωs2 = solar time angle at end of period [rad] (Equation 30)
Ra = ((12 * 60) / pi) * Gsc * dr * ((ωs2 - ωs1) * sin(φ) * cos(δ) + cos(φ) * cos(δ) * (sin(ωs2) - sin(ωs1)) ) # EUQATION 28

SyntaxError: invalid syntax (<ipython-input-28-70bd72aef421>, line 10)

In [29]:
# DAYLIGHT HOURS (N) [hours]
    # ωs = the sunset hour angle in radians given by Equation 25 or 26. Mean values for N
max_daylight_hour_duration = (24 / pi) * sunset_hour_angle # EQUATION 34
max_daylight_hour_duration

10.914524515584047

In [30]:
# SOLAR RADIATION (Rs) [MJ / m * day]
    # If the solar radiation, Rs, is not measured, it can be calculated with the Angstrom formula which relates solar
    # radiation to extraterrestrial radiation and relative sunshine duration:

    # Rs = solar or shortwave radiation [MJ / m * day]
    # n = actual duration of sunshine [hour]
    # N = maximum possible duration of sunshine or daylight hours [hour]
    # n/N = relative sunshine duration [-]
    # Ra = extraterrestrial radiation [MJ / m * day]
    # as = regression constant, expressing the fraction of extraterrestrial radiation reaching the earth on overcast days (n = 0),
    # as + bs = fraction of extraterrestrial radiation reaching the earth on clear days (n = N).
    # Rs is expressed in the above equation in [MJ / m * day]. The corresponding equivalent evaporation in [mm / day] is
    # obtained by multiplying Rs by 0.408 (Equation 20). Depending on atmospheric conditions (humidity, dust) and
    # solar declination (latitude and month), the Angstrom values a s and b s will vary. Where no actual solar radiation
    # data are available and no calibration has been carried out for improved as and bs parameters, the values as = 0.25 and bs = 0.50 are recommended.
    
    # RULE OF THUMBS
    # for cloudless days Rs circa 75% of Ra
    # for cloudy days    Rs circa 25% of Ra
    
global_radiation = (REGRESSION_CONSTANT + FRACTION_EXTRATERRESTRIAL_RADIATION * (measured_daylight_hour_duration / max_daylight_hour_duration)) * extraterrestrial_radiation # EQUATION 35
global_radiation

14.54973450096309

In [31]:
# the water lost in mm / day (water depths units) due to global radiation reaching the earth
global_radiation_mm = MJ_per_m2_day_to_mm_per_day(global_radiation)
global_radiation_mm

5.9362916763929405

In [32]:
# THIS IS ONLY TO CALCULATE THE MAXIMUM GLOBAL RADIATION CAN REACH THE EARTH.
#     the same formula is used in global radiation, but here (n = N) or (measured_daylight_length = max_daylight_length)
# CLEAR-SKY SOLAR RADIATION (Rso)
    # The calculation of the clear-sky radiation, Rso, when n = N, is required for computing net longwave radiation.
    # Rso = clear-sky solar radiation [MJ / m * day]
    # as + bs = fraction of extraterrestrial radiation reaching the earth on clear-sky days (n = N)
# --For near sea level or when calibrated values for as and bs are available --
global_radiation_clear_sky = (REGRESSION_CONSTANT + FRACTION_EXTRATERRESTRIAL_RADIATION) * extraterrestrial_radiation # EQUATION 36
global_radiation_clear_sky

18.969513096789083

In [33]:
# the water lost in mm / day (water depths units) due to global radiation with a clear sky reaching the earth
global_radiation_clear_sky_mm = MJ_per_m2_day_to_mm_per_day(global_radiation_clear_sky)
global_radiation_clear_sky_mm

7.739561343489945

In [34]:
# NET SOLAR OR NET SHORTWAVE RADIATION (Rns) [MJ / m * day]
    # Rns = net solar or shortwave radiation [MJ / pow(m, 2) * day]
    # αlbedo = albedo or canopy reflection coefficient, which is 0.23 for the hypothetical grass reference crop [dimensionless]
    # Rs = the incoming solar radiation [MJ / m * day]
    # Rns = is expressed in the above equation in [MJ / pow(m, 2) * day]
net_shortwave_radiation = (1 - albedo) * global_radiation # EQUATION 38
# the global radiation receive on earh minus the radiation that is reflected back by the albedo factor
net_shortwave_radiation

11.20329556574158

In [35]:
# the water lost in mm / day (water depths units) due to global radiation reaching the earth less the radiation that is reflected back to space - factor called albedo
net_shortwave_radiation_mm = MJ_per_m2_day_to_mm_per_day(net_shortwave_radiation)
net_shortwave_radiation_mm

4.570944590822564

In [36]:
# NET LONGWAVE RADIATION (Rnl) [MJ / m * day]
    # Rnl = net outgoing longwave radiation [MJ pow(m, 2) * day]
    # σ = Stefan-Boltzmann constant [4.903 10 -9 MJ / pow(K, 4) * pow(m, 2) * day]
    # T_max_k = maximum absolute temperature during the 24-hour period [K = °C + 273.16]
    # T_min_k = minimum absolute temperature during the 24-hour period [K = °C + 273.16]
    # ea = actual vapour pressure [kPa]
    # Rs / Rso = relative shortwave radiation (limited to ≤ 1.0)
    # Rs = measured or calculated. (Equation 35) solar radiation [MJ / pow(m, 2) * day]
    # Rso = calculated (Equation 36 or 37) clear-sky radiation [MJ / pow(m, 2) * day]
from math import sqrt
air_temp_max_in_kelvin = celcius_to_kelvin(air_temp_max)
air_temp_min_in_kelvin = celcius_to_kelvin(air_temp_min)
net_longwave_radiation = STEFAN_BOLTZMANN_CONSTANT * ((pow(air_temp_max_in_kelvin, 4) + pow(air_temp_min_in_kelvin, 4)) / 2) * (0.34 - 0.14 * sqrt(actual_vapour_pressure_rh)) * (1.35 * (global_radiation / global_radiation_clear_sky) - 0.35) # EQUATION 39
net_longwave_radiation

3.990658132475608

In [37]:
# NET RADIATION (Rn) [MJ / m * day]
    # The net radiation (R n ) is the difference between the incoming net shortwave radiation (R ns ) and the outgoing net longwave radiation (R nl )
net_radiation = net_shortwave_radiation - net_longwave_radiation # EQUATION 40
net_radiation

7.212637433265972

In [38]:
net_radiation_mm =  MJ_per_m2_day_to_mm_per_day(net_radiation)
# the amount of mm of water in a surface that is evaporated by the energy of the sun throughout a period of time per day that 
net_radiation_mm

2.9427560727725166

# STEP # 4
 
## SOIL ANALYSIS
---

* **SOIL HEAT FLUX (G)**

In [39]:
# SOIL HEAT FLUX (G)
    #
    # -- For day and ten-day periods --
    # As the magnitude of the day or ten-day soil heat flux beneath the grass reference surface is relatively
    # small, it may be ignored and thus:
G_day = 0 # EQUATION 42
    # -- for monthly periods
    # When assuming a constant soil heat capacity of 2.1 MJ m-3 °C-1 and an appropriate soil depth,
    # Equation 41 can be used to derive G for monthly periods:
G_month_next_previous = 0.07 * (air_temp_next_month - air_temp_previous_month)
    # if air_temp_next_month (1 month in the future of calculation) is not available then
G_month_current_previous = 0.14 * (air_temp_current_month - air_temp_previous_month)
    # -- For hourly or shorter periods --
    # For hourly (or shorter) calculations, G beneath a dense cover of grass does not correlate well with air
    # temperature. Hourly G can be approximated during DAYLIGHT periods as:
G_hour_day = 0.1 * net_radiation # EQUATION 45
    # and during NIGHTTIME periods as:
G_hour_night = 0.5 * net_radiation # EQUATION 46
    # Where the soil is warming, the soil heat flux G is positive. The amount of energy required for this
    # process is subtracted from R n when estimating evapotranspiration.
'G_day = {0}, G_month_next_previous = {1}, G_month_current_previous = {2}, G_hour_day = {3}, G_hour_night = {4}'.format(G_day, G_month_next_previous, G_month_current_previous, G_hour_day, G_hour_night)

'G_day = 0, G_month_next_previous = 0.3290000000000001, G_month_current_previous = 0.28000000000000025, G_hour_day = 0.7212637433265973, G_hour_night = 3.606318716632986'

In [40]:
soil_heat_flux = G_day
soil_heat_flux

0

# STEP # 5
 
## WIND ANALYSIS
---

**WIND PROFILE RELATIONSHIP**
* Wind speeds measured at different heights above the soil surface are different. Surface friction tends to slow down wind passing over it. Wind speed is slowest at the surface and increases with height.
* For this reason anemometers are placed at a chosen standard height, i.e.,10 m in meteorology and 2 or 3 m in agrometeorology. For the calculation of evapotranspiration, wind speed measured at 2m above the surface is required. To adjust wind speed data obtained from insruments placed atelevations other than the standard height of 2 m, a logarithmic wind speed profile may be used for measurements above a short grassed surface

1. u2 = wind speed at 2 m above ground surface [m / s]
2. uz = measured wind speed at z m above ground surface [m * s]
3. z = height of measurement above ground surface [m]
* The corresponding multipliers or conversion factors are given in Annex 2 (Table 2.9) and are plotted in Figure 16.

In [41]:
from math import log
wind_speed_at_2m = wind_speed_measurement * (4.87 / log(67.8 * wind_speed_measurement_height - 5.42))
# the wind measurements ideally is at 2m above soil, however if data with different height is used this convertor factor aids calculating its conversion FIGURE 16 FIGURE 16
# Conversion factor to convert wind speed measured at a certain height above ground level to wind speed at the standard height (2 m) 
wind_speed_at_2m # [m / s]

0.872342617516981

# STEP # FINAL
 
## EVAPOTRANSPIRATION CROP REFERENCE ANALYSIS

In [42]:
# INCORRECT
# AREODYNAMIC RESISTANCE (ra) [s / m]
    # ra = aerodynamic resistances [s / m]
    # wind_speed_measurement_height = height of wind measurements [m] meters
    # air_rh _measurement_height = height of humidity measurements [m] meters
    # d = zero plane displacement height [m] meters
    # Zom = roughness length governing momentum transfer [m] meters
    # Zoh = roughness length governing transfer of heat and vapour [m] meters
    # k = von Karman's constant, 0.41 [None]
    # Uz = wind speed at height z [m / s]
from math import log, pow
d = (2 / 3) * crop_height_ref
Zom = 0.123 * crop_height_ref
Zoh = 0.1 * Zom
aerodynamic_resistance = (log((wind_speed_measurement_height - d) / Zom) * log((air_rh_measurement_height - d) / Zoh)) / (pow(KARMAN_CONSTANT, 2) * wind_speed_measurement) # EQUATION 4
aerodynamic_resistance

261.8129625057894

In [43]:
# (BULK) SURFACE RESISTANCE (rs) [s / m]
    # rs = (bulk) surface resistances [s / m]
    # rl = bulk stomatal resistance of the well-illuminated leaf [s pow(m, -1)]
    # LAI_active = active (sunlit) leaf area index [pow(m, 2) (leaf area) pow(m, -2) (soil surface)] meters
        # The LAI values for various crops differ widely but values of 3-5 are common for many mature crops. For a given
        # crop, green LAI changes throughout the season and normally reaches its maximum before or at flowering
        # LAI further depends on the plant density and the crop variety.
        # A general equation for LAIactive is: LAI_active = 0.5 LAI
            # which takes into consideration the fact that generally only the upper half of dense clipped grass is actively
            # contributing to the surface heat and vapour transfer. For clipped grass a general equation for LAI is:
            # LAI = 24 * h
            # where h is the crop height [m] meter
            # The stomatal resistance, rl of a single leaf has a value of about 100 s m-1 under well-watered conditions
LAI = 24 * crop_height_ref
LAI_active = 0.5 * LAI
surface_resistance = STOMATAL_RESISTANCE / LAI_active # EQUATION 5
surface_resistance

69.44444444444444

In [45]:
# "A hypothetical reference crop with an assumed crop height of 0.12 m, a fixed surface resistance of 70 s m-1 and an albedo of 0.23."
#     used to arrrive at this particular formula ( considering wind and rh temp measurement at height of 2 meters and witha constant pressure etc.)
# REFERENCE CROP EVAPOTRANSPIRATION [mm / day]
    # ETo = reference evapotranspiration [mm / day]
    # Rn = net radiation at the crop surface [MJ / (pow(m, 2) * day]
    # G = soil heat flux density [MJ / (pow(m, 2) * day)]
    # T = mean daily air temperature at 2 m height [°C]
    # Uz = wind speed at z or 2 m height [m / s] # calculations should be done at 2 m height
    # es = saturation vapour pressure [kPa]
    # ea = actual vapour pressure [kPa]
    # es - ea = saturation vapour pressure deficit [kPa]
    # ∆ = slope vapour pressure curve [kPa / °C]
    # γ = psychrometric constant [kPa / °C]
        # Radiation is MJ / pow(m, 2) * day -> (converted to mm / day) = radiation / 2.45 = 0.408*radiation = [mm / day]
        # (∆ * (Rn - G) + Pa * Cp * ((es - ea) / ra)) / (∆ + γ * (1 + (rs / ra))) # EQUATION 3
evapotranspiration_ref = (0.408 * saturation_vapour_pressure_slope * (net_radiation - soil_heat_flux) + psychrometric_constant * (900 / (air_temp_mean + 273)) * wind_speed_measurement * (saturation_vapour_pressure - actual_vapour_pressure_rh)) / (saturation_vapour_pressure_slope + psychrometric_constant * (1 + 0.34 * wind_speed_measurement)) # EQUATION 6
evapotranspiration_ref
# NOTE: calculating the psychrometric contants for norther hemisphere it should be positive and for sourtther hemisphere it should be negative
# http://www.fao.org/docrep/X0490E/x0490e06.htm#TopOfPage

2.670933162914566